# Calculating PMFs with AWH in GROMACS <a id='another_cell'></a>
Here we learn how to calculate the potential of mean force
(PMF) along a reaction coordinate (RC) using the accelerated weight
histogram method (AWH) in GROMACS.  We will go through both how to set
up the input files needed, as well as how to extract and analyze the
output after having run the simulation. For more information about
the AWH method itself and how it can be used we refer to \cite{}[TODO how cite?].
What you need to know right now is that AWH applies a time-dependent bias
potential along the chosen RC, which is tuned during the simulation
such that it "flattens" the barriers in the PMF to improve sampling along the RC.
With better sampling, the PMF can be calculated more accurately than using unbiased MD. [TODO Movie/figure?]

*Author: Viveca Lindahl   
Email: vivecal@kth.se*

## Prerequisites
This tutorial assumes you have/know
* basic knowledge of using a Linux shell 
* basic usage of the molecular visualization tool VMD
* an installation of GROMACS 2018.1 (available as gmx in your PATH)
* python modules numpy, matplotlib

## The case study: DNA base pair opening
We will calculate a PMF for opening a DNA base
pair. The DNA double helix is a very stable structure. Opening a base pair
requires breaking hydrogen bonds between the bases and crossing a high free energy
barrier.  That's why we need to enhance the sampling by applying a bias!
<img src="figs/dna-helix.png" alt="dna" style="height: 300px;"/>
As our RC we use the distance between the two atoms forming the central hydrogen-bond the two bases in a pair. Let's have a look at the system and the reaction coordinate using VMD. The `-e` flag below tells VMD to excute the commands that are in the following tcl-script. These commands change how the system is visually represented. For instance, we have hidden all the water to better see the DNA itself, and we have put punchy colors on the atoms defining the RC of our target base pair. Now run this and VMD should pop up:

In [ ]:
!vmd visualization/dna-centered.gro -e visualization/representation.tcl

Rotate the structure and look for the two (nitrogen) atoms in green. The distance between these will serve as our RC for that base pair. Quit VMD to continue.

## The MD parameter (.mdp) file 
We'll now assume we have already built and equilibrated the system, so that we are almost ready to go. To use AWH we basically just need to add some extra parameters in the mdp file. Go to and check out the directory that has all the run files of our first AWH example:

In [ ]:
%cd awh-simple
!ls -l

Find the differences between the mdp file using AWH and an mdp file for a vanilla MD simulation:

In [ ]:
!diff no-awh.mdp awh.mdp

Here '<' refers to the content of the first argument to `diff` (the non-AWH mdp) and '>' to the second (AWH mdp). E.g., we increased the number of steps (`nsteps`) for AWH. The more relevant parameters are the ones prefixed `pull` and `awh`. What do these parameters mean? Generally, google "gromacs documentation" to find out, but the comments in the mdp file should be enough here.

## The index (.ndx) file
The .mdp file now depends on some definitions of atom groups; we need to have an index file for these. Here our groups are as simple as they get: each group contains a single nitrogen atom. But don't get tempted to edit an index file manually! The traditional tool to use is `gmx make_ndx` and a more general and powerful tool is `gmx select`. We focus on AWH here and provide the index file. Double-check that the pull groups (e.g. `pull-group1-name`) we saw in the mdp file are actually defined in the index file:


In [ ]:
!grep -A 1  N1orN3  index.ndx  # '-A 1' to show also 1 line after the match 

One atom per group looks right. In a real study, a better check would be to visualize the atom indices (e.g. with VMD).

## Starting the simulation
Here we won't run the actual simulation; a directory with data has already been provided. But for practice, we will generate a tpr, as usual with `grompp`. The flag `-quiet` tells `grompp` to be less verbose.

In [ ]:
!gmx grompp -quiet -f awh.mdp -n index.ndx -p topol.top -o topol.tpr

Note that the values of the pull coordinate values in the provided .gro file are printed by `grompp`. Does it look reasonable? 

## Check the log file
Now let's look at the data we've provided. Some information related to the AWH initial convergence can be found in the `mdrun` log file.

In [ ]:
!grep 'covering' data/md.log
!grep 'out of the' data/md.log

In the AWH intial stage, the free energy (and bias) update size is kept constant *during* each transition of the sampling interval, and decreased in a stepwise manner after the interval has been covered.  After exiting the initial stage, the free energy update size will decrease steadily with time. The initial stage is a type of "burn-in" process that improves robustness of the method. The idea is to first get a quick and rough estimate in the intitial stage, followed by refinement in the final stage. 

## The reaction coordinate trajectory
The trajectory of the pull coordinate is found in `pullx.xvg`. For a quick look at the contents of such files from the terminal we can use the interactive plotting tool `xmgrace`

In [ ]:
# -free for nice layout
!xmgrace -free data/pullx.xvg # wait for it... 

For more serious plotting and data analysis, one can write a small python function to read the xvg file.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function that reads data from an .xvg file
def read_xvg(fname):
    data=[]
    with open(fname) as f: 
        for line in f:
            # Lines with metadata or comments start with #, @
            if not line.startswith(("@","#")):
                data.append(np.array([float(s) for s in line.split()]))
    data = np.vstack(data)
    return data

We can for instance plot the covering times.

In [ ]:
# Read pullx.xvg file and plot a selected part of the trajectory
data = read_xvg("data/pullx.xvg")
t, x = data[:,0], data[:,1]
plt.plot(t, x)
ax = plt.gca()

# Tweak
plt.ylim([0.25, 0.65])
tend =10000
plt.xlim([0,tend])
plt.xlabel("time (ns)")
plt.ylabel("distance (nm)");
plt.title('RC trajectory');

# Find covering times and plot them
# Extract substrings using regular expressions
import re

covering_logs = !grep covering data/md.log
covering_times = [float(re.findall('t = (.+?) ps', log)[0]) for  log in covering_logs]
for tcover in covering_times:
    plt.plot([tcover,tcover],[0,1])


Verify that the covering times coincide roughly with the transition times in the trajectory. What we expect to see here is that the covering time is shorter initially, when the the bias fluctuates more, and then the transitions times increase as the bias stops changing. You should always check this trajectory and make sure that there are at least a few transitions after the initial stage. If not, you need to simulate longer or you might have an RC issue.

## The `gmx awh` tool and the `awh.xvg` files

Data directly related to AWH is extracted using `gmx awh`. The data is stored in the energy file, `ener.edr`. The default output is `awh_t<time>.xvg`; there will be one file for each AWH output time, or less if you use the `-skip` flag. Let's dump the output into a separate directory.

In [ ]:
!mkdir -p awh-data
!gmx awh -quiet -s topol.tpr -f data/ener.edr -more -kt -o awh-data/awh.xvg

Above, the `-more` flag tells `gmx awh` to more than the PMF from the AWH files, e.g. histograms. The default gmx units of energy is kJ/mol, but with the flag `-kT` we get units of kT.

Let's have a click look at the contents of one of these files (from time 50 ns) using `xmgrace`

In [ ]:
t=50000 # ps
# -nxy to plot multicolumn data
!xmgrace -free -nxy awh-data/awh_t{t}.xvg # wait for it... 

The legends in the plot should help understanding what data is being plotted. In more detail:
* *PMF*: the free energy estimate along the RC,  $\xi(x)$.
* *Coord bias*: the bias acting on  $\xi$. Note: $\xi$ is attracted to large values of the bias.
* *Coord distr*: histogram of  $\xi$.
* *Ref value distr*: probability weight histogram of reference coordinate value, $\lambda$.
* *Target ref value distr*: the target distribution for $\lambda$ (uniform by default).
* Friction metric: a metric for local friction, or 1/diffusion (here unitless). Can pinpoint hard-to-sample regions.

**Sanity checks for the distributions (zoom in to see them better)**
* *Coord distr* $\approx$ *Ref value distr*, or the force constant in the mdp file may is too weak there.

Note that for the smallest distances, where atoms are repelling each other and the PMF is steep, this doesn't hold entirely! But this is not an important sampling region for us. 

* *Ref value distr* $\approx$ *Target ref value distr*, i.e. that what we sampled is close to the distribution that we targeted. If not true, we may have a poor bias estimate/need to sample longer.

Let's look closer at the evolution of the PMF.

In [ ]:
# Plot PMFs from AWH xvg files

# A list of all AWH files
fnames = !ls awh-data/awh_t*xvg

# Extract time of each file
times = [float(re.findall('awh_t(.+?).xvg', fname)[0]) for fname in fnames]

# Sort the files chronologically
fnames = [f for (t,f) in sorted(zip(times, fnames))]
times.sort()
print "Number of files:", len(fnames)
print "Time in between files ", times[1] - times[0], 'ps'
print "Last time", times[-1], 'ps'

In [ ]:
# Plot the PMF from first files/times
labels=[]
istart = 0 # Start plotting this file index
nplot = 10  # Number of files to plot
for fname in fnames[istart:istart+nplot]:
    data=read_xvg(fname)
    labels.append(re.findall('awh_t(.+?).xvg', fname)[0] + ' ps') # use the time as label
    plt.plot(data[:,0], data[:,1])
plt.xlabel('distance (nm)')
plt.ylabel('PMF (kT)')
plt.xlim([0.25,0.60])
plt.ylim([0,20])
plt.legend(labels);

Try increasing the `istart` variable above to see how the PMF estimates are changing (less and less) over time. *Note*: this convergence does not easily translate into error estimates for the PMF. To get such error bars the simplest is to run multiple (independent) AWH simulations and calculate standard errors from that [add exercise?]

That's about what's needed for running a AWH simulation in GROMACS!

## Multiple bias sharing walkers
A simple way of parallelizing AWH is to run multiple simulations, using the `mdrun` flag `-multidir` and have them all share the collected samples and bias potential. This can reduce the length of the initial stage significantly. If we just apply `-multidir` we will get multiple *independent* simulations. We need to add a couple of things in the mdp file to make AWH share information across simulations

In [ ]:
%cd ../awh-multi;

In [ ]:
!diff ../awh-simple/awh.mdp awh-multi.mdp

These options tells `gmx mdrun` to share AWH biases across simulations when using `gmx mdrun -multidir`, and which AWH biases should be communicating with which (here there is only one bias, indexed by 1).